In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt

# Load the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00325/Sensorless_drive_diagnosis.txt'
data = pd.read_csv(url, sep=' ', header=None)

# Split data into features and labels
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Keras model
def create_model():
    model = Sequential()
    model.add(Dense(256, input_dim=48, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(11, activation='softmax'))
    optimizer = Adam(lr=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Use KerasClassifier to wrap the Keras model for Scikit-learn compatibility
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the parameter grid for grid search
param_grid = {
              'batch_size': [32, 64, 128],
              'epochs': [10, 20, 50],
              'optimizer': ['adam', 'rmsprop']
              }

# Define the Keras model
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(256, input_dim=48, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(11, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Use KerasClassifier to wrap the Keras model for Scikit-learn compatibility
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the parameter grid for grid search
param_grid = {
              'batch_size': [32, 64],
              'epochs': [10],
             }

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters and accuracy score from grid search
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Define the parameter distributions for random search
param_dist = {'batch_size': [32, 64, 128],
              'epochs': [10, 20, 50],
              'optimizer': ['adam', 'rmsprop']}

# Perform random search
random = RandomizedSearchCV(estimator=model, param_distributions=param_dist, cv=3, n_iter=10, random_state=42)
random_result = random.fit(X_train, y_train)

# Print the best parameters and accuracy score from random search
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

# Define the k-fold cross validation generator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Train the model using k-fold cross validation
cv_scores = []
for train_idx, test_idx in kfold.split(X_train, y_train):
    # Fit the model
    model = create_model()
    model.fit(X_train[train_idx], y_train[train_idx], epochs=20, batch_size=64, verbose=0)
    # Evaluate the model on the test fold
    scores = model.evaluate(X_train[test_idx], y_train[test_idx], verbose=0)
    cv_scores.append(scores[1] * 100)

# Print the mean and standard deviation of the k-fold cross validation scores
print("CV Scores: %.2f%% (+/- %.2f%%)" % (np.mean(cv_scores), np.std(cv_scores)))
